# Core Imports and Setup

In [1]:
import os
from pathlib import Path

import warnings
warnings.filterwarnings("ignore")

import logging
logging.getLogger("openff.toolkit").setLevel(logging.ERROR)

from openff import toolkit, evaluator

# 0) Registering Custom ThermoML Properties

In [2]:
from openff.units import unit
from openff.evaluator import properties
from openff.evaluator.datasets.thermoml import thermoml_property
from openff.evaluator.datasets import PhysicalProperty, PropertyPhase

@thermoml_property("Osmotic coefficient", supported_phases=PropertyPhase.Liquid)
class OsmoticCoefficient(PhysicalProperty):
    """A class representation of a osmotic coeff property"""

    @classmethod
    def default_unit(cls):
        return unit.dimensionless
    
...

custom_thermoml_props = [
    OsmoticCoefficient,
]

for custom_prop_cls in custom_thermoml_props:    
    setattr(properties, custom_prop_cls.__name__, custom_prop_cls)

# 1) - Loading ThermoML Data Sets

## Extracting data from ThermoML

In [3]:
from openff.evaluator.datasets import PhysicalProperty, PropertyPhase
from openff.evaluator.datasets.thermoml import thermoml_property
from openff.units import unit

from openff.evaluator.datasets.thermoml import ThermoMLDataSet

# data_set = ThermoMLDataSet.from_doi(
#     '10.1016/j.jct.2011.03.012',
#     # "10.1016/j.fluid.2013.10.034",
# )
data_set = ThermoMLDataSet.from_doi( # defaults from exaple notebook
    "10.1016/j.fluid.2013.10.034",
    "10.1021/je1013476",
)

## Filtering data set

In [4]:
from openff.evaluator.datasets.curation.components.filtering import FilterByPropertyTypes, FilterByPropertyTypesSchema
from openff.evaluator.datasets.curation.components.filtering import FilterByTemperature, FilterByTemperatureSchema
from openff.evaluator.datasets.curation.components.filtering import FilterByPressure, FilterByPressureSchema
from openff.evaluator.datasets.curation.components.filtering import FilterBySmiles, FilterBySmilesSchema

# Property
data_set = FilterByPropertyTypes.apply(
    data_set, FilterByPropertyTypesSchema(property_types=["Density"])
)

# Temperature
data_set = FilterByTemperature.apply(
    data_set, FilterByTemperatureSchema(minimum_temperature=298.0, maximum_temperature=330.0)
)

# Pressure
data_set = FilterByPressure.apply(
    data_set, FilterByPressureSchema(minimum_pressure=100.0, maximum_pressure=101.426)
)

# Solvent
data_set = FilterBySmiles.apply(
    data_set, FilterBySmilesSchema(smiles_to_include=["CCO", "CC(C)O"])
)

print(len(data_set))

4


In [5]:
pandas_data_set = data_set.to_pandas()
pandas_data_set[
    [
        "Temperature (K)",
        "Pressure (kPa)",
        "Component 1",
        "Density Value (g / ml)",
        "Source",
    ]
].head()

,Temperature (K),Pressure (kPa),Component 1,Density Value (g / ml),Source
0,298.15,101.325,CC(C)O,0.78270,10.1016/j.fluid.2013.10.034
1,298.15,101.325,CCO,0.78507,10.1021/je1013476
2,303.15,101.325,CCO,0.78072,10.1021/je1013476
3,313.15,101.325,CCO,0.77197,10.1021/je1013476


## Adding Extra Data

### Defining new properties

In [6]:
from openff.evaluator.datasets import MeasurementSource, PropertyPhase
from openff.evaluator.substances import Substance
from openff.evaluator.thermodynamics import ThermodynamicState
from openff.evaluator.properties import EnthalpyOfVaporization

from openff.units import unit

# define thermo state
thermodynamic_state = ThermodynamicState(
    temperature=298.15 * unit.kelvin, pressure=1.0 * unit.atmosphere
)

# define compounds 
ethanol = Substance.from_components("CCO")
isopropanol = Substance.from_components("CC(C)O")

# define source of measurements
source = MeasurementSource(doi="10.1016/S0021-9614(71)80108-8")

# define measurement values
ethanol_hvap = EnthalpyOfVaporization(
    thermodynamic_state=thermodynamic_state,
    phase=PropertyPhase.Liquid | PropertyPhase.Gas,
    substance=ethanol,
    value=42.26 * unit.kilojoule / unit.mole,
    uncertainty=0.02 * unit.kilojoule / unit.mole,
    source=source,
)
isopropanol_hvap = EnthalpyOfVaporization(
    thermodynamic_state=thermodynamic_state,
    phase=PropertyPhase.Liquid | PropertyPhase.Gas,
    substance=isopropanol,
    value=45.34 * unit.kilojoule / unit.mole,
    uncertainty=0.02 * unit.kilojoule / unit.mole,
    source=source,
)
data_set.add_properties(ethanol_hvap, isopropanol_hvap)

### Inspecting and saving new properties

In [7]:
# save for future use
data_set_path = Path('filtered_data_set.json')
data_set.json(data_set_path, format=True)

# inspect new properties
pandas_data_set = data_set.to_pandas()
pandas_data_set

,Id,Temperature (K),Pressure (kPa),Phase,N Components,Component 1,Role 1,Mole Fraction 1,Exact Amount 1,EnthalpyOfVaporization Value (kJ / mol),EnthalpyOfVaporization Uncertainty (kJ / mol),Density Value (g / ml),Density Uncertainty (g / ml),Source
0,6d4bd208a79f4952915846bc62faa32e,298.15,101.325,Liquid,1,CC(C)O,Solvent,1.0,None,NaN,NaN,0.78270,0.000075,10.1016/j.fluid.2013.10.034
1,dc19418156f144eaae3ff345219850f6,298.15,101.325,Liquid,1,CCO,Solvent,1.0,None,NaN,NaN,0.78507,0.000060,10.1021/je1013476
2,4a7ee1bf6df44930b67bb0920bbcd14c,303.15,101.325,Liquid,1,CCO,Solvent,1.0,None,NaN,NaN,0.78072,0.000060,10.1021/je1013476
3,046aea953ee04b038ad96902ff9a0f03,313.15,101.325,Liquid,1,CCO,Solvent,1.0,None,NaN,NaN,0.77197,0.000065,10.1021/je1013476
4,5e3f6067d2ec43438b91658a8562387f,298.15,101.325,Liquid + Gas,1,CCO,Solvent,1.0,None,42.26,0.02,NaN,NaN,10.1016/S0021-9614(71)80108-8
5,e27e1d14d10e401c9831dff74ed06ad6,298.15,101.325,Liquid + Gas,1,CC(C)O,Solvent,1.0,None,45.34,0.02,NaN,NaN,10.1016/S0021-9614(71)80108-8


# 2) Estimating Data Sets

## Loading data set and FF parameters

In [8]:
from openff.evaluator.datasets import PhysicalPropertyDataSet
from openff.evaluator.forcefield import SmirnoffForceFieldSource

# load data
data_set_path = Path('filtered_data_set.json')
data_set = PhysicalPropertyDataSet.from_json(data_set_path)

# load FF
force_field_path = "openff-1.0.0.offxml"
force_field_source = SmirnoffForceFieldSource.from_path(force_field_path)

## Defining Calculation Schemas

In [9]:
from openff.evaluator.properties import Density, EnthalpyOfVaporization
from openff.evaluator.client import RequestOptions

density_schema = Density.default_simulation_schema(n_molecules=256)
h_vap_schema = EnthalpyOfVaporization.default_simulation_schema(n_molecules=256)

# Create an options object which defines how the data set should be estimated.
estimation_options = RequestOptions()

# Specify that we only wish to use molecular simulation to estimate the data set.
estimation_options.calculation_layers = ["SimulationLayer"]

# Add our custom schemas, specifying that the should be used by the 'SimulationLayer'
estimation_options.add_schema("SimulationLayer", "Density", density_schema)
estimation_options.add_schema("SimulationLayer", "EnthalpyOfVaporization", h_vap_schema)

## Launching a Server and Client

In [13]:
from openff.evaluator.backends import ComputeResources
from openff.evaluator.backends.dask import DaskLocalCluster
from openff.evaluator.server import EvaluatorServer
from openff.evaluator.client import EvaluatorClient
from openff.evaluator.client import ConnectionOptions

# define client to submit queries
port = 8119
evaluator_client = EvaluatorClient(ConnectionOptions(server_port=port))

# define available / preferred resources
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
resources = ComputeResources(
    number_of_threads=1,
    number_of_gpus=1,
    preferred_gpu_toolkit=ComputeResources.GPUToolkit.CUDA,
)

with DaskLocalCluster(number_of_workers=1, resources_per_worker=resources) as calculation_backend:
    # spin up server
    evaluator_server = EvaluatorServer(calculation_backend=calculation_backend, delete_working_files=False, port=port)
    evaluator_server.start(asynchronous=True)

    # estimate data set by submitting calculation schemas to newly-created server
    request, exception = evaluator_client.request_estimate(
        property_set=data_set,
        force_field_source=force_field_source,
        options=estimation_options,
    )

    # Wait for the results.
    results, exception = request.results(synchronous=True, polling_interval=30)
    assert exception is None

INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at: inproc://172.21.16.71/366477/23
INFO:distributed.scheduler:  dashboard at:  http://172.21.16.71:8787/status
INFO:distributed.scheduler:Registering Worker plugin shuffle
INFO:distributed.worker:      Start worker at: inproc://172.21.16.71/366477/26
INFO:distributed.worker:         Listening to:         inproc172.21.16.71
INFO:distributed.worker:          Worker name:                          0
INFO:distributed.worker:         dashboard at:         172.21.16.71:39391
INFO:distributed.worker:Waiting to connect to: inproc://172.21.16.71/366477/23
INFO:distributed.worker:-------------------------------------------------
INFO:distributed.worker:              Threads:                          1
INFO:distributed.worker:               Memory:                  31.00 GiB
INFO:distributed.worker:      Local Directory: /tmp/dask-scratch-space/worker-dksba2ug
INFO:distributed.worker:------------------------------------

In [14]:
a = results.estimated_properties.json("estimated_data_set.json", format=True)

In [15]:
print(a)

{
  "@type": "openff.evaluator.datasets.datasets.PhysicalPropertyDataSet",
  "properties": [
    {
      "@type": "openff.evaluator.properties.density.Density",
      "gradients": [],
      "id": "773858b70c18451e817c70f611f8cd79",
      "phase": 2,
      "source": {
        "@type": "openff.evaluator.datasets.provenance.CalculationSource",
        "fidelity": "SimulationLayer",
        "provenance": "{\"protocol_schemas\": [{\"id\": \"773858b70c18451e817c70f611f8cd79|decorrelate_trajectory\", \"type\": \"DecorrelateTrajectory\", \"inputs\": {\".allow_merging\": true, \".time_series_statistics\": {\"full_path\": \"b749b50ecaae494394be40d2aaf9cef9|conditional_group/773858b70c18451e817c70f611f8cd79|average_density.time_series_statistics\", \"@type\": \"openff.evaluator.workflow.utils.ProtocolPath\"}, \".input_coordinate_file\": {\"full_path\": \"b749b50ecaae494394be40d2aaf9cef9|conditional_group/b749b50ecaae494394be40d2aaf9cef9|production_simulation_liquid.output_coordinate_file\", \"@ty